In [2]:
# importing libraries
from sklearn.linear_model import LogisticRegression as LR
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn import model_selection as msl
import numpy as np
import pandas as pd


In [3]:
# importing data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# cleaning data
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
features2 = ['Pclass','Sex','Age','SibSp','Parch','Embarked']
Y = train_data['Survived']
X_train = train_data[features]
X_test = test_data[features]

for col in X_train.columns:
    if X_train[col].dtypes == 'object':
        d = {}
        itr = 0
        for j in range(len(X_train[col])):
            if X_train[col].iloc[j] in d:
                X_train[col].iloc[j] = d[X_train[col].iloc[j]]
            else:
                itr += 10
                d[X_train[col].iloc[j]] = itr
                X_train[col].iloc[j] = d[X_train[col].iloc[j]]
        for j in range(len(X_test[col])):
            if X_test[col].iloc[j] in d:
                X_test[col].iloc[j] = d[X_test[col].iloc[j]]
            else:
                itr += 10
                d[X_test[col].iloc[j]] = itr
                X_test[col].iloc[j] = d[X_test[col].iloc[j]]



for col in X_train.columns:
    if X_train[col].isnull().sum():
        a = X_train[col].mode()
        cnt = 0
        b = 0
        for i in a:
            b += i
            cnt += 1
        b = b//cnt
        X_train[col].fillna(b,inplace = True)

for col in X_test.columns:
    if X_test[col].isnull().sum():
        a = X_test[col].mode()
        cnt = 0
        b = 0
        for i in a:
            b += i
            cnt += 1
        b = b//cnt
        X_test[col].fillna(b,inplace = True)

X_test.isnull().sum()

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [11]:
# generating model 
X2 = X_train.copy()
X2_t,X2_v, Y2_t, Y2_v = msl.train_test_split(X_train,Y,random_state = 1 )

alf = LR(max_iter = 10000, C = 2, multi_class = 'ovr',class_weight='auto' )
algo = RFC(random_state = 2,max_depth = 3 )
alf.fit(X2_t,Y2_t)
algo.fit(X2_t,Y2_t)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=2, verbose=0,
                       warm_start=False)

In [14]:
print(alf.score(X2_v,Y2_v))
print(algo.score(X2_v,Y2_v))

0.7982062780269058
0.7937219730941704


In [13]:
# converting to output file
alf.fit(X_train,Y)
algo.fit(X_train,Y)
Y_ans = algo.predict(X_test)
ans = {
    'PassengerId':test_data['PassengerId'],
    'Survived':Y_ans
}
ans = pd.DataFrame(ans)
ans.to_csv('preds.csv',index = False)